# Perplexity computation

In [1]:
import math
from collections import defaultdict
from typing import List

from gensim.utils import simple_preprocess

### Defining the Training and Test Data

We start by creating a **small training corpus**, which consists of a few simple sentences.  
This corpus will serve as the "knowledge base" from which our model learns word probabilities.  

Next, we define a **test sentence**.  
Perplexity will be computed on this sentence to measure how well the probability distribution learned from the training corpus predicts unseen text.


In [2]:
# Sample training corpus demonstrating perplexity computation
training_corpus = [
    "the cat sat on the mat",
    "the dog ran in the park", 
    "a cat likes fish",
    "the mat is soft"
]

# Test sentence for perplexity calculation
test_sentence = "the cat likes the mat"

### Building a Bigram Model

This block:
- Tokenizes sentences with `<s>` and `</s>` markers.  
- Counts **unigrams** (single words) and **bigrams** (word pairs).  
- Prints the counts to show what the model has learned.  

We also tokenize the test sentence so it’s ready for probability and perplexity calculation later.


In [4]:
# Tokenize and add special tokens
def tokenize_with_boundaries(sentence: str) -> List[str]:
    # TODO: return list of tokens with <s> at the start and </s> at the end
    return ['<s>'] + sentence.split() + ['</s>']  # Replace with actual tokenization logic
    pass

# Build bigram counts
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

# Tokenizing and counting
print(f"Tokenized corpus")
for sentence in training_corpus:
    tokens = tokenize_with_boundaries(sentence)
    print(f"\tTokens: {tokens}")
    
    # TODO: count unigrams
    for token in tokens:
        unigram_counts[token] += 1
    
    # TODO: count bigrams
    for i in range(len(tokens) - 1):
        bigram = (tokens[i], tokens[i + 1])
        bigram_counts[bigram] += 1

print(f"\nUnigram counts:")
for word, count in sorted(unigram_counts.items()):
    print(f"   {word}: {count}")

print(f"\nBigram counts:")
for bigram, count in sorted(bigram_counts.items()):
    print(f"   {bigram}: {count}")

test_tokens = tokenize_with_boundaries(test_sentence)
print(f"\nTokenized test sentence: {test_tokens}")

Tokenized corpus
	Tokens: ['<s>', 'the', 'cat', 'sat', 'on', 'the', 'mat', '</s>']
	Tokens: ['<s>', 'the', 'dog', 'ran', 'in', 'the', 'park', '</s>']
	Tokens: ['<s>', 'a', 'cat', 'likes', 'fish', '</s>']
	Tokens: ['<s>', 'the', 'mat', 'is', 'soft', '</s>']

Unigram counts:
   </s>: 4
   <s>: 4
   a: 1
   cat: 2
   dog: 1
   fish: 1
   in: 1
   is: 1
   likes: 1
   mat: 2
   on: 1
   park: 1
   ran: 1
   sat: 1
   soft: 1
   the: 5

Bigram counts:
   ('<s>', 'a'): 1
   ('<s>', 'the'): 3
   ('a', 'cat'): 1
   ('cat', 'likes'): 1
   ('cat', 'sat'): 1
   ('dog', 'ran'): 1
   ('fish', '</s>'): 1
   ('in', 'the'): 1
   ('is', 'soft'): 1
   ('likes', 'fish'): 1
   ('mat', '</s>'): 1
   ('mat', 'is'): 1
   ('on', 'the'): 1
   ('park', '</s>'): 1
   ('ran', 'in'): 1
   ('sat', 'on'): 1
   ('soft', '</s>'): 1
   ('the', 'cat'): 1
   ('the', 'dog'): 1
   ('the', 'mat'): 2
   ('the', 'park'): 1

Tokenized test sentence: ['<s>', 'the', 'cat', 'likes', 'the', 'mat', '</s>']


### Bigram Probabilities with Add-1 Smoothing

Now we compute bigram probabilities.  
To avoid zero probabilities for unseen word pairs, we use **add-1 smoothing** (Laplace smoothing).  

For each bigram in the test sentence:
- The probability is `(count(w1, w2) + 1) / (count(w1) + |V|)`.  
- We also compute `log2(prob)` for use in perplexity calculation.


In [6]:
len(bigram_counts)

21

In [7]:
vocab_size = len(unigram_counts)

def get_bigram_prob_smoothed(w1: str, w2: str) -> float:
    """Get bigram probability with add-1 smoothing"""
    # TODO: Implement formula (hint: slide 41)
    numerator = bigram_counts[(w1, w2)] + 1
    denominator = unigram_counts[w1] + vocab_size
    return numerator / denominator

print(f"Computing the Bigram probabilities:")
log_prob_sum = 0
n_tokens = len(test_tokens) - 1  # Number of bigrams

for i in range(len(test_tokens) - 1):
    w1, w2 = test_tokens[i], test_tokens[i + 1]
    # TODO: compute probability and log2(prob)
    prob = get_bigram_prob_smoothed(w1, w2)
    log_prob = math.log2(prob)
    log_prob_sum += log_prob
    
    print(f"   P({w2}|{w1}) = ({bigram_counts[(w1, w2)]} + 1) / ({unigram_counts[w1]} + {vocab_size}) = {prob:.4f}")
    print(f"   log2({prob:.4f}) = {log_prob:.4f}\n")

Computing the Bigram probabilities:
   P(the|<s>) = (3 + 1) / (4 + 16) = 0.2000
   log2(0.2000) = -2.3219

   P(cat|the) = (1 + 1) / (5 + 16) = 0.0952
   log2(0.0952) = -3.3923

   P(likes|cat) = (1 + 1) / (2 + 16) = 0.1111
   log2(0.1111) = -3.1699

   P(the|likes) = (0 + 1) / (1 + 16) = 0.0588
   log2(0.0588) = -4.0875

   P(mat|the) = (2 + 1) / (5 + 16) = 0.1429
   log2(0.1429) = -2.8074

   P(</s>|mat) = (1 + 1) / (2 + 16) = 0.1111
   log2(0.1111) = -3.1699



### Perplexity Computation

Finally, we calculate **perplexity** of the test sentence under our bigram model:  

\begin{align}
\text{Perplexity} = 2^{-\frac{1}{N} \sum \log_2 P(w_i \mid w_{i-1})}
\end{align}

- `N` = number of bigrams in the test sentence.  
- We average the log probabilities and exponentiate to get perplexity.  

A **lower perplexity** means the model finds the test sentence more predictable.


In [9]:
print(f"\nPerplexity calculation:")
print(f"   Sum of log probabilities: {log_prob_sum:.4f}")
print(f"   Number of tokens (N): {n_tokens}")

# TODO: Compute perplexity (hint: equation above + slide 30)
perplexity = 2 ** (-log_prob_sum / n_tokens)
print(f"   Perplexity = {perplexity:.4f}")


Perplexity calculation:
   Sum of log probabilities: -18.9489
   Number of tokens (N): 6
   Perplexity = 8.9269


## MCQ

### 4.1. Definition of Perplexity  

What does perplexity measure in language models?  

A. The total number of words in the test set<br>  
B. The unpredictability or "surprise" of a model when predicting text<br>  
C. The size of the vocabulary<br>  
D. The average frequency of bigrams<br>  

**Answer:** 

---

### 4.2. Formula for Perplexity  

Which of the following is the correct formula for perplexity of a test set with *N* tokens?  

A. $\text{Perplexity} = \frac{1}{N} \sum \log P(w_i)$<br>  
B. $\text{Perplexity} = 2^{-\frac{1}{N} \sum \log_2 P(w_i \mid context)}$<br>  
C. $\text{Perplexity} = \prod_{i=1}^{N} P(w_i)$<br>  
D. $\text{Perplexity} = N^{\sum P(w_i)}$<br>  

**Answer:** 

---

### 4.3. Interpretation of Perplexity  

If a model has **lower perplexity** on a test set, what does it mean?  

A. The model is more confident and better at predicting the test data<br>  
B. The model is overfitting<br>  
C. The vocabulary size is smaller<br>  
D. The training corpus is too simple<br>  

**Answer:** 

---

### 4.4. Smoothing and Perplexity  

Why is **add-1 smoothing (Laplace smoothing)** used when computing perplexity?  

A. To reduce the vocabulary size<br>  
B. To ensure unseen word pairs do not get zero probability<br>  
C. To increase the average log probability<br>  
D. To make the model faster to train<br>  

**Answer:**
